In [ ]:
from __future__ import print_function
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pickle
from astropy.io import fits

# path to the spectrum data files (change if you are getting errors loading data)
path=''

### astr3400 day 1 notebook: analyzing a quasar spectrum from SDSS-V

#### 1. load spectrum file 

this is a single spectrum file from one fiber of one observation from SDSS-V
the data are in FITS format, commonly used by astronomers

In [ ]:
file_name = path+'spec-9391-58072-0898.fits'
f = fits.open(file_name)

In [ ]:
# brief look at what's in the file
# it is mostly in the form of "binary tables" a flexible way of storing different types of data
print(f.info()) 
print(f[2].columns)

In [ ]:
# define spectral data we will use
# this is flux in (complicated) units of 10^-17 erg cm^-2 s^-1 Angstrom^-1
wavelength = 10**f[1].data['loglam']
flux = f[1].data['flux']

# plot the spectrum
plt.plot(wavelength,f[1].data['flux'])
plt.ylim(0,80); plt.xlim(3500,10000)
plt.xlabel(r'wavelength (Angstrom)',fontsize=12)
plt.ylabel(r'flux ($10^{-17}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)',fontsize=12)

#### 2. estimate the color temperature of the spectrum

Wien's displacement law says that for a blackbody spectrum 

$\lambda_{\rm max} = \frac{b}{T_c}$

with a constant $b$ and a temperature that we call the "color" temperature $T_c$ in astronomy

In [ ]:
# estimate \lambda_max in angstroms and use it to get T_c
lambda_max = 4500.

# this value of b is correct for a wavelength in angstroms
b = 2.8978e7

# put in the correct expression for Tc
Tc = b/lambda_max

print('color temperature in Kelvin: ',Tc)

#### 3. we often convert the width of an emission line (Delta lambda) to a velocity using the Doppler effect:

$\frac{\Delta \lambda}{\lambda_0} = \frac{v}{c}$

where $\lambda_0$ is the central wavelength of the emission line.

#### for the emission line near 6500 Angstroms, what would the velocity v be in units of km/s?

In [ ]:
# zoomed in spectrum near the emission line of interest
plt.plot(wavelength,f[1].data['flux'])
plt.ylim(0,50); plt.xlim(4500,5500)
plt.xlabel(r'wavelength (Angstrom)',fontsize=12)
plt.ylabel(r'flux ($10^{-17}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)',fontsize=12)

In [ ]:
# estimated width of broad emission line near 5000 Angstroms in units of Angstroms 
# (you can do this by eye)
delta_lam = 100.

# central wavelength of emission line
lambda0 = 4800.

# calculate velocity
c_kms = 3e5

v = delta_lam/lambda0*c_kms

print('velocity width of emission line in km/s: ',v)

#### 4. this broadening can arise from random thermal motions in hot gas according to: 

$v \approx \sqrt{\frac{k T}{m}}$

using $m$ as the mass of a proton, what temperature T would be needed to explain your width from part 4?


does this T match the color temperature you derived in part 3?

As a result, do you think the width of the emission lines can be explained by random thermal motions ("thermal broadening")?

In [ ]:
# mass of a proton in g
m = 1.67e-24

# Boltzmann constant k in cgs units
k = 1.38e-16

T = m*(v*1e5)**2./k

print('temperature for thermal broadening: ', T)

#### 5. let's figure out the redshift of the quasar using its emission lines

the function below will plot the line locations at a given redshift z 
(0 is present day, most distant quasars seen in optical light are at z ~ 1-4)

your job is to find the redshift where the locations of the expected emission lines match the observed ones!

(if you have the right redshift, the solid black lines on the plot should match the strongest lines in the spectrum and the dashed lines should also land on emission lines. the default z = 0 is not a good match!)

### here are some useful atomic emission lines in quasar spectra (units of Angstrom, same as our spectra):
O VI 1034

Ly $\alpha$ 1216

C IV 1549

C III 1908

Mg II 2799

H$\beta$ 4862 

H$\gamma$ 4361

H$\alpha$ 6564

In [ ]:
# the strong atomic transitions are called Lyman alpha, Carbon IV, Magnesium II, H beta, H alpha
# they are from hydrogen, carbon, magnesium
strong_lines=[1216.,1549.,2799.,4862.,4960.,5008.,6564.]
weak_lines=[1240.,1400.,1665.,1908.,3727.,4103.,4342.]
def plot_z(z):
    for s in strong_lines:
        plt.plot(s*(1.+z)+np.zeros(50),np.arange(50)/49.*(np.max(f[1].data['flux'])),color='k',linestyle='-')
    for w in weak_lines:
        plt.plot(w*(1.+z)+np.zeros(50),np.arange(50)/49.*(np.max(f[1].data['flux'])),color='k',linestyle='--')
        
# plotting and units
plt.plot(wavelength,f[1].data['flux'])
plt.ylim(0,75); plt.xlim(3500,10000)
plt.xlabel(r'wavelength (Angstrom)',fontsize=12)
plt.ylabel(r'flux ($10^{-17}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)',fontsize=12)

# try figuring out the redshift by changing the value here (it's > 0 and < 4!)
z = 0.
plot_z(z)

#### now plot the spectrum with the x-axis units of wavelength at the redshift of the quasar (the "rest frame")

In [ ]:
plt.plot((wavelength)/(1.+z),f[1].data['flux'])
plt.ylim(0.,70)
plt.xlabel(r'rest frame wavelength (Angstrom)',fontsize=12)
plt.ylabel(r'flux ($10^{-17}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)',fontsize=12)

now let's convert the light curve to a luminosity (energy radiated by the quasar per unit time)

the radiated luminosity at some frequency $\nu L_\nu$ is related to the observed flux $F_{\lambda_{\rm obs}}$ by:

$\nu L_\nu = 4 \pi D_L^2 \lambda_{\rm obs} F_{\lambda_{\rm obs}}$

where the subscript obs means observed frame and no subscript means emitted frame, and $D_L$ is the "luminosity distance"

#### what type of light was this when it was emitted by the quasar? (e.g. radio, infrared, optical, ultraviolet, X-ray?)

enter answer here

In [ ]:
# calculate the luminosity distance for our quasar redshift
from astropy.cosmology import WMAP9 as cosmo
dl=np.array(cosmo.luminosity_distance(z),dtype=float)
# D_L is in units of Mpc
# measure F_\lambda_obs at your choice of \lambda_obs where we are trying to measure the 
# flat continuum level and not the emission lines

lambdaobs = 6000.

# enter the value of the flat part of the spectrum near 6000 Angstroms
flambdaobs = XX

continuum_luminosity = 4.*np.pi*(dl*3.09e24)**2.*flambdaobs*lambdaobs
print(continuum_luminosity)

#### 6. how does the luminosity of the quasar compare to that of the Sun ($4\times10^{33}$ erg /s)? how about to that of the Milky Way galaxy ($5\times10^{43}$ erg/s)?

enter answer here